In [1]:
from bs4 import BeautifulSoup
import requests
import re
import json
import pandas as pd

In [2]:
url='https://mediabiasfactcheck.com/filtered-search/'
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

In [3]:
#content.find(text='getData()')
scripts = soup.find_all('script')
data = scripts[34] #34 is script that includes the JSON file that we want

In [4]:
for elem in soup(text=re.compile("function getData()")):
    data = elem.split("current_json = ", 2)
    data = data[2].split("updateFilters()")
    data = data[0].split(";")
jsondata = data[0] #data[0] contains the json file we are looking for 

In [5]:
result = json.loads(json.dumps(jsondata))
df = pd.DataFrame.from_dict(eval(result), orient='index') #have to do eval(result) because in this case, result isn't the dictionary itself, it's a string representation of a dictionary – eval makes it into dictionary form 
df.head(15)

,b,d,h,L,n,r,u,c
100percentfedup.com,Questionable Sources,100percentfedup.com,https://100percentfedup.com,"97,906",100% Fed up,Low,https://mediabiasfactcheck.com/100-percent-fed...,United States
1010wins.radio.com,Least Biased,1010wins.radio.com,https://1010wins.radio.com,0,1010 WINS AM,High,https://mediabiasfactcheck.com/1010-wins-am/,United States
www.10news.one,Questionable Sources,www.10news.one,https://www.10news.one,"2,011",10News.one,Low,https://mediabiasfactcheck.com/10news-one/,United States
12news.com,Least Biased,12news.com,https://12news.com,0,12 News KPNX,High,https://mediabiasfactcheck.com/12-news-kpnx/,United States
,Left,,https://,0,Washington Daily Wire,Mixed,https://mediabiasfactcheck.com/washington-dail...,United States
www.whitehouse.gov,Right,www.whitehouse.gov,https://www.whitehouse.gov,"13,742,825",1600 Daily,Mostly Factual,https://mediabiasfactcheck.com/1600-daily/,United States
21stcenturywire.com,Conspiracy-Pseudoscience,21stcenturywire.com,https://21stcenturywire.com,"332,291",21st Century Wire,Mixed,https://mediabiasfactcheck.com/21st-century-wire/,United States
us24news.com,Questionable Sources,us24news.com,https://us24news.com,527,24 News,Low,https://mediabiasfactcheck.com/24-news/,United States
247newsmedia.com,Questionable Sources,247newsmedia.com,https://247newsmedia.com,213,247 News Media,Very Low,https://mediabiasfactcheck.com/247-news-media/,Macedonia
secondamendmentdaily.com,Questionable Sources,secondamendmentdaily.com,https://secondamendmentdaily.com,0,2nd Amendment Daily News,Low,https://mediabiasfactcheck.com/2nd-amendment-d...,United States


In [6]:
df = df.rename(columns={"b": "Bias", "d": "URL", "h": "Full URL", "L": "References", "n": 
                        "Name", "r": "Factualness", "u": "MBFC analysis", "c": "Country"})
df.head()

,Bias,URL,Full URL,References,Name,Factualness,MBFC analysis,Country
100percentfedup.com,Questionable Sources,100percentfedup.com,https://100percentfedup.com,"97,906",100% Fed up,Low,https://mediabiasfactcheck.com/100-percent-fed...,United States
1010wins.radio.com,Least Biased,1010wins.radio.com,https://1010wins.radio.com,0,1010 WINS AM,High,https://mediabiasfactcheck.com/1010-wins-am/,United States
www.10news.one,Questionable Sources,www.10news.one,https://www.10news.one,"2,011",10News.one,Low,https://mediabiasfactcheck.com/10news-one/,United States
12news.com,Least Biased,12news.com,https://12news.com,0,12 News KPNX,High,https://mediabiasfactcheck.com/12-news-kpnx/,United States
,Left,,https://,0,Washington Daily Wire,Mixed,https://mediabiasfactcheck.com/washington-dail...,United States


In [7]:
print (df['Bias'].unique()) #looking at all the different bias labels

array(['Questionable Sources', 'Least Biased', 'Left', 'Right',
       'Conspiracy-Pseudoscience', 'Right-Center', 'Left-Center',
       'Satire', 'Pro-Science'], dtype=object)

In [9]:
#numerical value for each bias 
bias_dictionary = {"Left": -2, "Left-Center": -1, "Least Biased": 0, "Right-Center": 1, "Right": 2, 
                  "Questionable Sources": 0, "Conspiracy-Pseudoscience": 5, "Satire": -20, "Pro-Science": -5}
df['Bias Rating'] = df['Bias'].map(bias_dictionary)
df.head(20)


,Bias,URL,Full URL,References,Name,Factualness,MBFC analysis,Country,Bias Rating
100percentfedup.com,Questionable Sources,100percentfedup.com,https://100percentfedup.com,"97,906",100% Fed up,Low,https://mediabiasfactcheck.com/100-percent-fed...,United States,0
1010wins.radio.com,Least Biased,1010wins.radio.com,https://1010wins.radio.com,0,1010 WINS AM,High,https://mediabiasfactcheck.com/1010-wins-am/,United States,0
www.10news.one,Questionable Sources,www.10news.one,https://www.10news.one,"2,011",10News.one,Low,https://mediabiasfactcheck.com/10news-one/,United States,0
12news.com,Least Biased,12news.com,https://12news.com,0,12 News KPNX,High,https://mediabiasfactcheck.com/12-news-kpnx/,United States,0
,Left,,https://,0,Washington Daily Wire,Mixed,https://mediabiasfactcheck.com/washington-dail...,United States,-2
www.whitehouse.gov,Right,www.whitehouse.gov,https://www.whitehouse.gov,"13,742,825",1600 Daily,Mostly Factual,https://mediabiasfactcheck.com/1600-daily/,United States,2
21stcenturywire.com,Conspiracy-Pseudoscience,21stcenturywire.com,https://21stcenturywire.com,"332,291",21st Century Wire,Mixed,https://mediabiasfactcheck.com/21st-century-wire/,United States,5
us24news.com,Questionable Sources,us24news.com,https://us24news.com,527,24 News,Low,https://mediabiasfactcheck.com/24-news/,United States,0
247newsmedia.com,Questionable Sources,247newsmedia.com,https://247newsmedia.com,213,247 News Media,Very Low,https://mediabiasfactcheck.com/247-news-media/,Macedonia,0
secondamendmentdaily.com,Questionable Sources,secondamendmentdaily.com,https://secondamendmentdaily.com,0,2nd Amendment Daily News,Low,https://mediabiasfactcheck.com/2nd-amendment-d...,United States,0
